# Tworzenie wątków

Aby nasza symulacja mogła płynnie wyświetlać się w GUI, musimy uruchomić obliczenia związane z ruchem zwierzątek w tle. Nie możemy po prostu wstrzymywać naszego głównego wątku i co jakiś czas przesuwać zwierzątko, bo wtedy UI przestałby być responsywny. Trzeba pamiętać, że w aplikacjach graficznych główny wątek nieustannie rysuje nam na ekranie kontrolki. Każda interakcja, przesunięcie okienka myszką itp. wymaga odrysowania kontrolek. W momencie uśpienia takiego wątku GUI wyglądałoby jakby się zawiesiło (sprawdźcie to!). 

Rozwiązaniem jest oczywiście puszczenie całej symulacji w osobnym wątku, który co jakiś czas będzie się budził i aktualizował dane wyświetlane przez GUI. Nie jest to specjalnie trudne zadanie, bo nasz `SimulationEngine` już wcześniej realizował identyczny interfejs `Runnable` - a to właśnie ten interfejs trzeba zaimplementować by móc go opakować w wątek. Dla porządku możemy dodać deklarację `Runnable` do naszego interfejsu `IEngine` (choć wcale nie jest to potrzebne, jeśli skorzystamy potem z wyrażenia lambda):

```java
public interface IEngine extends Runnable {
}
```

Potem w miejscu tworzenia wątku wystarczy opakować symulację w `Thread`:
```java
SimulationEngine engine = new SimulationEngine(map, animalStartingPositions, directions);
Thread engineThread = new Thread(engine);
engineThread.start();
```

W naszym przykładzie mamy dość prostą sytuację, bo wątki nie walczą ze sobą o żadne zasoby. Trzeba jednak pamiętać, że w bardziej złożonych sytuacjach będziemy musieli zadbać o synchronizację lub projektować aplikację tak by wyeliminować wszelkie sekcje krytyczne (patrz wykład).

# Łączenie GUI z modelem

Stosunkowo sporo problemów sprawiło Wam prawidłowe powiązanie `SimulationEngine` z `App`. Najczęstszym rozwiązaniem było po prostu przekazanie instancji `App` do konstruktora `SimulationEngine`, a potem wywoływanie na niej bezpośrednio aktualizacji w każdym obrocie pętli symulacji. Oczywiście takie rozwiązanie zadziała, ale w dłuższej perspektywie sprawiłoby nam problemy. Dlaczego? Odpowiedź taka jak zawsze: bardzo ogranicza to możliwości swobodnego rozszerzania aplikacji. Zwróćmy uwagę, że `SimulationEngine`, podobnie jak klasy mapy i zwierzątek są centralną częścią naszej aplikacji, tzw. modelem. Ich zadaniem jest realizowanie "logiki biznesowej" naszego programu. Każdy inny moduł tego programu będzie korzystał z ich rezultatów więc można powiedzieć, że "wszyscy znają model". Gdyby jednak działało to w dwie strony i "model znałby wszystkich" to każda zmiana lub rozbudowa jednego z pozostałych komponentów wymuszałaby modyfikację modelu... a co za tym idzie, pewnie i innych modułów. Utrzymanie takiej aplikacji na dłuższą metę jest bardzo trudne. Dlatego powinniśmy ograniczać zależności w taki sposób by **wszyscy znali model, ale model nie znał nikogo**. 

A przynajmniej nie bezpośrednio. W naszym przypadku chcemy przecież by symulacja informowała widok, że zmieniły się pozycje zwierzątek. Jak to zrobić? Ponownie wykorzystując nasz ulubiony wzorzec projektowy, obserwator.

Mamy tu tak naprawdę dwie możliwości: 
1) Stworzyć zupełnie nowy interfejs obserwatora, np. `SimulationListener` z metodą  `simulationChanged()`. Jego obsługa w `SimulationEngine` wyglądałaby identycznie jak realizacja `IPositionChangedObserver` w klasie `Animal`.
2) Wykorzystac istniejący `IPositionChangedObserver` i rejestrować obserwatora na wszystkich zwierzątkach z symulacji:

    ```java
    public void  addObserver(IPositionChangeObserver observer) {
        map.getAnimals().forEach(animal -> animal.addObserver(observer));
    }
    ```
    Tutaj uwaga: w GUI chcemy pokazywać nie tylko zmiany pozycji, ale również orientacji zwierzątek więc konieczne byłoby rozbudowanie interfejsu, np. tak:
    ```java
    public interface IPositionChangeObserver {

        void positionChanged(Vector2d oldPosition, Vector2d newPosition);

        void orientationChanged(MapDirection oldOrientation, MapDirection newOrientation);
    }
    ```

Oba rozwiązania będą tutaj działać (drugie nieco mniej wydajnie, ale można je jeszcze zoptymalizowac łącząc dwie metody powiadamiające w jedną). Oba pozwolą też uniezależnić model od GUI - zwróćcie uwagę, że obecnie `SimulationEngine` nie wie już nic na temat `App`. Wie tylko, że ma kolekcję *jakichś* obserwatorów symulacji i nic więcej. Gdybyśmy np. zmienili GUI napisane w JavieFX na inną bibliotekę, nie musielibyśmy dokonywać **żadnych** zmian w `SimulationEngine` i reszcie modelu.

**I na koniec ważna uwaga**: przykład z tej laborki pokazuje w uproszczony sposób, jak separować model od widoku. Nie jest to jednak perfekcyjne rozwiązanie. W takich przypadkach powinniśmy tak naprawdę oprzeć architekturę aplikacji o wzorzec **Model View Controller (MVC)**. On również korzysta z obserwatorów, ale dokłada jeszcze jedną warstwę między widokiem a  modelem. Osoby, które wybrały ścieżkę inżynierską na studiach z Informatyki spotkają się z tym wzorcem na trzecim roku (i jeszcze będą go miały dość). Warto o nim [jednak poczytać już teraz](https://pl.wikipedia.org/wiki/Model-View-Controller), może uda się go przestować w którymś z projektów.

## `runLater()`

Gdy nasz wątek symulacyjny wywołuje metodę odświeżającą widok to ona również wykona się w wątku symulacyjnym. Może to doprowadzić do bardzo niebezpiecznej sytuacji, w której próbujemy rysować kontrolki z wielu wątków jednocześnie. Generalnie **tylko główny wątek graficzny powinien wykonywać jakiekolwiek operacje na kontrolkach**. Pamiętajmy, że główny wątek nieustannie pracuje, odrysowując w kółko całe okno. Gdy wtrąci się w to rysowanie wątek symulacyjny, wytworzymy klasyczny *race condition*, tyle że naszym "współdzielonym zasobem" będą właśnie kontrolki na ekranie. Może to poskutować powstaniem tzw. gliczy i artefaktów w oknie aplikacji. Żeby się przed tym uchronić powinniśmy stosować "przełączkę", która wrzuca logikę związaną z GUI na kolejkę głównego wątku rysującego:
```java
public void simulationChanged() {
    Platform.runLater(() -> {   
        refreshMap();
    });
}
```

## ...i jeszcze więcej obserwatorów

W tym zadaniu po raz pierwszy pojawiła się też interakcja uzytkownika naszej aplikacji z GUI, czyli przycisk do startowania symulacji. W przypadku przycisku naturalne wydaje się, że w reakcji na jego kliknięcie COŚ powinno się stać. COŚ, a dokładniej jakiś fragment kodu. Implementacja takiego zachowania wygląda całkiem intuicyjnie:
```java
 startButton.setOnAction(event -> {
            Thread engineThread = createEngineThread();
            engineThread.start();
 });
```
Oczywiście mamy tu po raz kolejny wyrażenie lambda, a wiemy już że kryją się pod nimi interfejsy z pojedynczymi metodami. W tym przypadku takim interfejsem jest `EventHandler` (można to podglądnąć najeżdżając kursorem na strzałkę lambdy w IntelliJ, albo wejść do środka klikając ją z wciśniętym Ctrl):
```java
public interface EventHandler<T extends Event> extends EventListener {
    void handle(T event);
}
```
Jest on parametryzowany typem `T`, żeby móc go stosować dla różnych rodzajów zdarzeń w GUI, ale tak poza tym to zwyczajny interfejs. Klasa przycisku trzyma zapewne atrybut reprezentujący obiekt takiego interfejsu ustawiany setterem `setOnAction()`, a następnie, gdy przycisk zostaje kliknięty, powiadamia handler wołajać metodę `handle()`... Mam nadzieję, że na tym etapie brzmi to wszystko już wystarczająco znajomo - po raz kolejny mamy tu wzorzec obserwator, tym razem jednak dostarczony przez bibliotekę Javy, a nie nasz własny. W tym przypadku użycie tego wzorca jest wręcz konieczne, bo przecież klasa przycisku nie zna klas, które samodzielnie piszemy żeby dostarczyć obsługę kliknięć.

# Liskov Substitution Principle (LSP)

Mimo że laborka koncentrowała się wokół wątków i stworzenia kontrolek w UI, udało się tu przemycić jeden bardzo klasyczny problem, na który natknęli się niektórzy z Was. W klasie `GuiElementBox` mieliśmy stworzyć wizualną reprezentację elementu mapy (trawy lub zwierzątka). Składała się ona z ikonki oraz podpisu. W przypadku ikonki rozwiązanie zostało zasugerowane, ale podpis trzeba było zaimplementować samodzielnie. Wiemy tylko, że powinien być inny dla trawy (napis "Grass") niż dla zwierzątek (pozycja). `GuiElementBox` otrzymuje jednak ogólną reprezentację elementu, czyli `IMapElement`. Pomysł, który od razu przychodzi tutaj do głowy to sprawdzenie typu elementu:

```java
String name;
if (element instanceof Animal) {
    name = element.getPosition().toString();
}
else {
    name = "Grass";
}
Label label = new Label(name);
```
Jeśli tak wygląda Wasze rozwiązanie to uważajcie, bo właśnie zdenerwowaliście panią Barbarę Liskov, autorkę zasady LSP, która mówi: 

*Funkcje, które korzystają z referencji bądź wskaźników do klas bazowych, muszą być w stanie poprawnie używać obiektów klas dziedziczących po tych klasach bazowych, bez ich dokładnej znajomości.*

Jakie to ma konsekwencje w praktyce i czemu należy słuchać p. Liskov? Ponownie problemem może się tu okazać rozszerzalność. Jeśli dołożymy do naszej mapy w przyszłości nowy typ elementu, np. zombie-zwierząta to będziemy musieli pamiętać o tym by zmodyfikować również klasę `GuiElementBox`. Jeśli tego nie zrobimy to... nic się nie stanie, program najpewniej się skompiluje i uruchomi. Problem pojawi się wtedy, gdy dojdzie on do drabinki ifów powyżej próbując wyświetlić podpis dla zombie-zwierzaka. Dla kodu powyżej skończy on na `else` i błędnie opisze zombiaka jako trawę. Często takie sytuacje kończą się też rzuceniem `NullPointerException` (jeśli drabinka nie przewiduje else'a). 

Jak można więc napisać to wszystko lepiej? Podobnie jak w przypadku obrazków, możemy dorzucić do klasy `IMapElement` jeszcze jedną metodę, np. `getDescription()`. Wtedy opisy elementów byłyby dostarczone przez model:
```java
public class Animal implements IMapElement {
    ...
    @Override
    public String getDescription() {
        return getPosition().toString();
    }
}

public class Grass implements IMapElement {
    ...
    @Override
    public String getDescription() {
        return "Grass";
    }
}
```
Zwróćmy uwagę, że dodanie kolejnej klasy, np. `ZombieAnimal` wymusi na nas dodanie implementacji `getDescription()`, bo inaczej program się nie skompiluje! Nie ma więc możliwości, że o czymkolwiek zaponimy. 

Z kolei w `GuiElementBox` wszystko znacznie się uprości:

```java
String name = element.getDescription();
Label label = new Label(name);
```
